In [1]:
import os
import random
import shutil
import re
from datetime import datetime
import piexif
import config

In [2]:
file_list = []
unique_file_types = set()

for root, dirs, files in os.walk(config.INPUT_FILEPATH):
    for file in files:
        file_list.append(file)

        _, file_extension = os.path.splitext(file)
        unique_file_types.add(file_extension.lower())

random_files = random.sample(file_list, 50)
with open('sampled_files.txt', 'w') as file:
    file.write('\n'.join(random_files))

unique_file_types

{'.jpg'}

In [3]:
regex_pattern = r'(\d\d?)-(\d\d?)-(\d{4})'

for root, dirs, files in os.walk(config.INPUT_FILEPATH):
        # Create corresponding directories in the output directory
        relative_path = os.path.relpath(root, config.INPUT_FILEPATH)
        output_root = os.path.join(config.OUTPUT_FILEPATH, relative_path)
        os.makedirs(output_root, exist_ok=True)

        # Create the 'undated' subdirectory if it doesn't exist
        undated_folder_path = os.path.join(output_root, 'undated')
        os.makedirs(undated_folder_path, exist_ok=True)

        # Iterate through all files in the current directory
        for file in files:
            file_path = os.path.join(root, file)
            exif_dict = piexif.load(file_path)

            # Check if the file name matches the regex pattern
            match = re.search(regex_pattern, file)
            if match:
                # Copy the file
                output_file_path = os.path.join(output_root, file)
                shutil.copy2(file_path, output_file_path)

                # Extract the content of the regex match
                date_month = int(match.group(1))
                date_day = int(match.group(2))
                date_year = int(match.group(3))

                new_date = datetime(date_year, date_month, date_day, 0, 0, 0).strftime("%Y:%m:%d %H:%M:%S")
                exif_dict['Exif'][piexif.ExifIFD.DateTimeOriginal] = new_date
                exif_bytes = piexif.dump(exif_dict)
                piexif.insert(exif_bytes, output_file_path)
            else:
                # Copy the file to the 'undated' subdirectory
                undated_file_path = os.path.join(undated_folder_path, file)
                print(file_path)
                print(undated_file_path)
                shutil.copy2(file_path, undated_file_path)


E:\family photos test\Willa (Curtis) Morrison with her children (Richard) Dick & James 12-1955.jpg
E:\family photos modified\.\undated\Willa (Curtis) Morrison with her children (Richard) Dick & James 12-1955.jpg
E:\family photos test\Back Lf-Rt-Verble Morey, Edna Morey, Eva Morey, Louise Jones, Ruth Jones, Gilbert Morey, Edra Morey Tonsfeldt, Arney Tonsfeldt, Bill Morey Front row-Martin Tonsfeldt, Janis Morey, David Morey & Merrill Tonsfeldt 1950 something.jpg
E:\family photos modified\.\undated\Back Lf-Rt-Verble Morey, Edna Morey, Eva Morey, Louise Jones, Ruth Jones, Gilbert Morey, Edra Morey Tonsfeldt, Arney Tonsfeldt, Bill Morey Front row-Martin Tonsfeldt, Janis Morey, David Morey & Merrill Tonsfeldt 1950 something.jpg


FileNotFoundError: [WinError 3] The system cannot find the path specified